<a href="https://colab.research.google.com/github/KozinIlya/Test-Study/blob/main/analiz_voronki%2003.02.22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Проверить, отличается ли время прохождения различных этапов для пользователей, которые прошли обучение,
 от пользователей, не начинавших обучение. Насколько обучение сокращает время прохождения этапов?

2. Посмотреть, существует ли зависимость между вероятностью оплаты вопросов и количеством обучений,которые начинал
 или завершал пользователь. Нужно доказать, что успешное обучение само по себе влияет на оплату, и без разницы, каким этапом оно шло.

3. Дополнительная задача. Как часто пользователи начинают обучение после того, как они выбрали уровень сложности? 
 Это позволит нам понять, насколько прозрачен процесс взаимодействия с игрой: если пользователи после выбора уровня
 сложности обращаются к обучению, значит, работа с приложением непонятна.

In [1]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
'подгтовка данных для дальнейшего анализа'
import pandas as pd
import io
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

df = pd.read_csv(io.StringIO(uploaded['7_4_Events.csv'].decode('utf-8')))
cond = (df.start_time>='2018-01-01') & (df.start_time<='2018-12-31') & (df.event_type=='registration')
registered = df[cond]['user_id'].to_list()
events = df[df.user_id.isin(registered)]                                          #датафрейм событий для пользователей 2018г
events.start_time = pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')

df1 = pd.read_csv(io.StringIO(uploaded['purchase.csv'].decode('utf-8')))
purchase_df = df1[df1.user_id.isin(registered)]                            #датафрейм оплат для пользователей 2018г      
purchase_df['event_type'] = 'purchase'
purchase_df.event_datetime = pd.to_datetime(purchase_df.event_datetime, format='%Y-%m-%dT%H:%M:%S')

events = events.rename(columns={'id':'event_id'})
purchase_df = purchase_df.rename(columns={'id':'purchase_id'})

ws = events.event_type == 'tutorial_start'
wf = events.event_type =='tutorial_finish'
start_list = events[ws]['user_id'].to_list()     #список пользователей начинавших обучение
dont_start_list = list(set(registered) - set(start_list)) #список пользователей не начинавших обучение
finish_list = events[wf]['user_id'].to_list()                 #список пользователей прошедших обучение
dont_finish_list = list(set(start_list) - set(finish_list)) #спиок пользователей не завершивших обучение


In [ ]:
# Гипотеза 1

'найти среднее время прохождения этапов'

'группа не начавшие обучение'

purchase_dont_start = purchase_df[purchase_df.user_id.isin(dont_start_list)]
dont_start_reg = events[events.user_id.isin(dont_start_list)][events['event_type'] == 'registration']
dont_start_merge = dont_start_reg.merge(purchase_dont_start, on='user_id',how='inner')
dont_start_merge['deltatime_purchase'] = (dont_start_merge['event_datetime'] - dont_start_merge['start_time'])/pd.Timedelta('1 hour')
dont_start_lc = events[events.user_id.isin(dont_start_list)][events['event_type'] == 'level_choice'].rename(columns={'start_time': 'level_choice_datetime'})
dont_start_merge1 = dont_start_reg.merge(dont_start_lc, on='user_id',how='inner')
dont_start_merge1['deltatime_level_choice'] = (dont_start_merge1['level_choice_datetime'] - dont_start_merge1['start_time'])/pd.Timedelta('1 hour')
dont_start_pc = events[events.user_id.isin(dont_start_list)][events['event_type'] == 'pack_choice'].rename(columns={'start_time': 'pack_choice_datetime'})
dont_start_merge2 = dont_start_lc.merge(dont_start_pc, on='user_id',how='inner')
dont_start_merge2['deltatime_pack_choice'] = (dont_start_merge2['pack_choice_datetime'] - dont_start_merge2['level_choice_datetime'])/pd.Timedelta('1 hour')
dont_start_merge3 = dont_start_pc.merge(purchase_dont_start, on='user_id',how='inner')
dont_start_merge3['deltatime_pack_purch'] = (dont_start_merge3['event_datetime'] - dont_start_merge3['pack_choice_datetime'])/pd.Timedelta('1 hour')
'группа прошедших обучение'
purchase_finish = purchase_df[purchase_df.user_id.isin(finish_list)]
finish_reg = events[events.user_id.isin(finish_list)][events['event_type'] == 'registration']
finish_merge = finish_reg.merge(purchase_finish, on='user_id',how='inner')
finish_merge['deltatime_purchase1'] = (finish_merge['event_datetime'] - finish_merge['start_time'])/pd.Timedelta('1 hour')
finish_lc = events[events.user_id.isin(finish_list)][events['event_type'] == 'level_choice'].rename(columns={'start_time': 'level_choice_datetime'})
finish_merge1 = finish_reg.merge(finish_lc, on='user_id',how='inner')
finish_merge1['deltatime_level_choice1'] = (finish_merge1['level_choice_datetime'] - finish_merge1['start_time'])/pd.Timedelta('1 hour')
finish_pc = events[events.user_id.isin(finish_list)][events['event_type'] == 'pack_choice'].rename(columns={'start_time': 'pack_choice_datetime'})
finish_merge2 = finish_lc.merge(finish_pc, on='user_id',how='inner')
finish_merge2['deltatime_pack_choice1'] = (finish_merge2['pack_choice_datetime'] - finish_merge2['level_choice_datetime'])/pd.Timedelta('1 hour')
finish_merge3 = finish_pc.merge(purchase_finish, on='user_id',how='inner')
finish_merge3['deltatime_pack_purch1'] = (finish_merge3['event_datetime'] - finish_merge3['pack_choice_datetime'])/pd.Timedelta('1 hour')



In [ ]:
'обьеденить данные для построения графика'

not_tutorial_purchase_delta = dont_start_merge[['user_id', 'deltatime_purchase']]
not_tutorial_level_choice_delta = dont_start_merge1[['user_id','deltatime_level_choice']]
not_tutorial_pack_choice_delta = dont_start_merge2[['user_id','deltatime_pack_choice']]
not_tutorial_pack_purch_delta = dont_start_merge3[['user_id','deltatime_pack_purch']]
dont_start_users = dont_start_reg.merge(not_tutorial_purchase_delta, on='user_id',how='left').merge(not_tutorial_level_choice_delta, on='user_id',how='left').merge(not_tutorial_pack_choice_delta, on='user_id',how='left').merge(not_tutorial_pack_purch_delta, on='user_id',how='left')

tutorial_purchase_delta = finish_merge[['user_id', 'deltatime_purchase1']]
tutorial_level_choice_delta = finish_merge1[['user_id','deltatime_level_choice1']]
tutorial_pack_choice_delta = finish_merge2[['user_id','deltatime_pack_choice1']]
tutorial_pack_purch_delta = finish_merge3[['user_id','deltatime_pack_purch1']]
tutorial_users = finish_reg.merge(tutorial_purchase_delta, on='user_id',how='left').merge(tutorial_level_choice_delta, on='user_id',how='left').merge(tutorial_pack_choice_delta, on='user_id',how='left').merge(tutorial_pack_purch_delta, on='user_id',how='left')

tu = tutorial_users[['event_type','user_id','deltatime_purchase1','deltatime_level_choice1','deltatime_pack_choice1','deltatime_pack_purch1']]

un_df = dont_start_users.merge(tu, on = ('user_id','event_type'), how = 'outer')

delta_df = un_df.groupby(by = 'event_type')['deltatime_purchase','deltatime_level_choice','deltatime_pack_choice','deltatime_pack_purch', 'deltatime_purchase1','deltatime_level_choice1','deltatime_pack_choice1','deltatime_pack_purch1'].mean().reset_index()

#оформление графика
fig = plt.figure()
fig, axes = plt.subplots(1, 4, figsize=(19,6))
axes[0].bar(x = 1, height = delta_df['deltatime_purchase'], 
         width=0.1, align = 'edge', label = 'не проходили обучение')
axes[0].bar(x = 1, height = delta_df['deltatime_purchase1'], 
         width=-0.1, align = 'edge', label = 'Прошли обучение')
axes[0].set_title('от регистрации до оплаты')
axes[0].legend(loc = 3)

axes[1].bar(x = 1, height = delta_df['deltatime_level_choice'], 
         width=0.1, align = 'edge', label = 'Не проходили обучение')
axes[1].bar(x = 1, height = delta_df['deltatime_level_choice1'], 
         width=-0.1, align = 'edge', label = 'Прошли обучение')
axes[1].set_title('от регистрации до  уровня сложности')
axes[1].legend(loc = 3)
axes[2].bar(x = 1, height = delta_df['deltatime_pack_choice'], 
         width=0.1, align = 'edge', label = 'Не проходили обучение')
axes[2].bar(x = 1, height = delta_df['deltatime_pack_choice1'], 
         width=-0.1, align = 'edge', label = 'Прошли обучение')
axes[2].set_title('от уровня сложности до бесплатного пакета')
axes[2].legend(loc = 3)
axes[3].bar(x = 1, height = delta_df['deltatime_pack_purch'], 
         width=0.1, align = 'edge', label = 'Не проходили обучение')
axes[3].bar(x = 1, height = delta_df['deltatime_pack_purch1'], 
         width=-0.1, align = 'edge', label = 'Прошли обучение')
axes[3].set_title('от бесплатного пакета до оплаты')
axes[3].legend(loc = 3)
#вывод данных
print('                 Сравнение среднего времени затраченного пользователями не проходившими обучения и пользователями закончившими обучение')
print('среднее время от регистрации до оплаты для пользователей не начавших обучение:  ', (dont_start_merge['deltatime_purchase'].mean())*(pd.Timedelta('1 hour')))
print('среднее время от регистрации до оплаты для пользователей закончивших обучение:  ', (finish_merge['deltatime_purchase1'].mean())*(pd.Timedelta('1 hour')))
print()
print('среднее время от регистрации до выбора уровня сложности для пользователей не начавших обучение:  ', (dont_start_merge1['deltatime_level_choice'].mean())*(pd.Timedelta('1 hour')))
print('среднее время от регистрации до выбора уровня сложности для пользователей прошедших обучение:   ', (finish_merge1['deltatime_level_choice1'].mean())*(pd.Timedelta('1 hour')))
print()
print('среднее время от выбора уровня сложности до выбора бесплатного пакета для пользователей не начавших обучение:  ', (dont_start_merge2['deltatime_pack_choice'].mean())*(pd.Timedelta('1 hour')))
print('среднее время от выбора уровня сложности до выбора бесплатного пакета для пользователей прошедших обучение:   ', (finish_merge2['deltatime_pack_choice1'].mean())*(pd.Timedelta('1 hour')))
print()
print('среднее время от выбора бесплатного пакета до оплаты для пользователей не начавших обучение:  ', (dont_start_merge3['deltatime_pack_purch'].mean())*(pd.Timedelta('1 hour')))
print('среднее время от выбора бесплатного пакета до оплаты для пользователей прошедших обучение:   ', (finish_merge3['deltatime_pack_purch1'].mean())*(pd.Timedelta('1 hour')))
percent = (1 - (finish_merge3['deltatime_pack_purch1'].mean()/dont_start_merge3['deltatime_pack_purch'].mean()))
print ('сокращение среднего времени, прошедшего от выбора бесплатного пакета до оплаты, у пользователей закончивших обучение:  {:.2%}'.format(percent))


In [ ]:
# гипотеза 2
purchase_users = purchase_df['user_id'].nunique()
'прошедшие обучение'
purchase_finish_users = purchase_finish['user_id'].nunique()
finish_users = events[wf]['user_id'].nunique()
pecent_p_f = purchase_finish_users/finish_users
print ('процент пользователей совершивших оплату среди прошедших обучение:    {:.2%}'.format(pecent_p_f))

'не завершившие обучение'
dont_finish_reg = events[events.user_id.isin(dont_finish_list)]['user_id'].nunique()
dont_finish_purchase = purchase_df[purchase_df.user_id.isin(dont_finish_list)]['user_id'].nunique()
pecent_p_d_f = dont_finish_purchase/dont_finish_reg
print ('процент пользователей совершивших оплату среди не завершивших  обучение: {:.2%}'.format(pecent_p_d_f))

'не проходившие обучение'
purchase_dont_start_users = purchase_dont_start['user_id'].nunique()
d_s_users = dont_start_reg['user_id'].nunique()
pecent_p_d_s =  purchase_dont_start_users/d_s_users
print ('процент пользователей совершивших оплату среди не проходивших обучение: {:.2%}'.format(pecent_p_d_s))

print()
#Построение графика
vals = [purchase_finish_users, purchase_dont_start_users, dont_finish_purchase]
labels = ["прошли", "не проходили", "не закончили"]
fig, ax = plt.subplots()
ax.pie(vals, labels=labels, autopct='%1.1f%%', wedgeprops=dict(width=0.25))
ax.axis("equal")
ax.set_title('Доля от общего числа пользователей, оплативших пакеты, в зависимости от прохождения обучения')


In [ ]:
#гипотеза 3
#обучение можно проходить несколько раз и возвращаться уже после выбора уровня сложности. Поэтому надо найти время первого выбора уровня(это время когда пользователь первый раз начал играл) и соотнести его со всеми записями времени начала обучения для каждого пользователя.
from datetime import timedelta

event_start = events[events['event_type'] == 'tutorial_start'].sort_values(by = 'start_time').rename(columns={'start_time':'tutorial_start_time'})
event_level = events[events['event_type'] == 'level_choice'].sort_values(by = 'start_time').drop_duplicates(subset=['user_id'], keep='first').rename(columns={'start_time':'level_choice_time'})

events_merge = event_start.merge(event_level, on='user_id',how='inner')


d = (events_merge['tutorial_start_time'] - events_merge['level_choice_time'])

tutorial_start_first = 0
level_choice_first = 0
for i in d:
  s = [i]
  for p in s:
    k = p.days       # чтобы сравнить deltatime с числом. если значение "дни" меньше 0 сначала пользователь начинал обучение а потом выбирал уровень сложности. Если нет-после начала игры пользователь возвращался к обучению.
    if k<0:
      tutorial_start_first += 1
    elif k>=0:
      level_choice_first += 1
print('количество пользователей, прошедших обучение до выбора уровня сложности:  ', tutorial_start_first)
print ('количество пользователей не проходивших обучение:  ', ((events['user_id'].nunique()) - tutorial_start_first))
print()
print('количество обучений  после начала игры: ', level_choice_first)
print()
# построение графика
vals = [tutorial_start_first, ((events['user_id'].nunique()) - tutorial_start_first),]
labels = ["начинали обучение", "не проходили обучение",]
fig, ax = plt.subplots()
ax.pie(vals, labels=labels, autopct='%1.1f%%', wedgeprops=dict(width=0.25))
ax.axis("equal")
ax.set_title('Соотношение числа пользователей ')


**Выводы:**



**Гипотеза №1**

Обучение сокращает время прохождения всех этапов кроме этапа "от регистрации до выбора уровня сложности", но это объясняется тем, что на этом этапе пользователи тратят время на обучение.
   Важно что пользователи прошедшие обучение на 11.43% быстрее переходят от бесплатный к платным пакетам.




Гипотеза №2

  процент пользователей совершивших оплату среди прошедших обучение:    14.13%

процент пользователей совершивших оплату среди не завершивших  обучение: 8.15%

процент пользователей совершивших оплату среди не проходивших обучение: 0.27%

  Среди пользователей приобретавших платные пакеты только 1.4% не проходили обучение. В то время как 90.4% оплативших прошли обучение полностью
   Таким образом однозначно можно считать что прохождение обучения положительно влияет на вероятность оплаты.




Гипотеза №3

  после начала игры пользователи 2792 раз возвращались к обучению. это достаточно большая цифра так как всего обучение прошли 9890 пользователей (пользователей для которых выбор уровня сложности был раньше первого обучения нет)





  Общий вывод:

  Пользователи прошедшие обучение часто проходят его повторно, а пользователи пропускающие обучение доходят до оплаты только в 0.27% случаев.
  
   Следует доработать процесс взаимодействия пользователя с игрой. Это позволит увеличить вовлеченность игроков (прежде всего тех, кто предпочитает пропускать процесс обучения), что должно повысит процент пользователей совершающих оплату.  
   Для более детального анализа следует проверить отличаются ли время взаимодействия с игрой у пользователей прошедших и непрошедших обучение